In [1]:
from pynucastro.rates import Library, RateFilter
from pynucastro.nucdata import Nucleus, BindingTable
from pynucastro.networks import PythonNetwork, Composition

In [2]:
full_lib = Library("reaclib_default2_20220329")

In [3]:
# Some other options include: ge64, te108
endpoint = Nucleus('ni56')

In [4]:
# Could introduce pp-chain nuclei (d, t, he3, be7, li7, etc.)

core_nuclei = ["p", "d", "he3", "he4", "li7", "be7", "be8", "b8", "c12",
               "n13", "n14", "n15", "o14", "o15", "o16", "o17", "o18",
               "f17", "f18", "f19", "f20", "ne18", "ne19", "ne20", "ne21"]
core_nuclei = list(map(Nucleus, core_nuclei))

In [5]:
core_lib = full_lib.linking_nuclei(core_nuclei)

In [6]:
def is_beta_plus(rate):
    """ Filter for beta+ decays (and electron captures). """
    
    if len(rate.products) != len(rate.reactants):
        return False
    if len(rate.reactants) != 1:
        return False
    
    react, = rate.reactants
    prod, = rate.products
    return prod.Z < react.Z

In [7]:
# Restrict the library to these 7 reaction types

# Forward rates
p_gamma = RateFilter(reactants="p", max_products=1, exact=False)
alpha_gamma = RateFilter(reactants="he4", max_products=1, exact=False)
alpha_p = RateFilter(reactants="he4", products="p", exact=False)

# Reverse and weak rates
gamma_p = RateFilter(products="p", max_reactants=1, exact=False)
gamma_alpha = RateFilter(products="he4", max_reactants=1, exact=False)
p_alpha = RateFilter(reactants="p", products="he4", exact=False)
beta_plus = RateFilter(filter_function=is_beta_plus)

# Compute reduced library
red_lib = full_lib.filter((p_gamma, alpha_gamma, alpha_p,
        gamma_p, gamma_alpha, p_alpha, beta_plus))

In [8]:
bintable = BindingTable()

In [9]:
from collections import deque

def flatten(iterable):
    """ Take iterable of iterables, and flatten it to one dimension. """
    
    for col in iterable:
        
        for item in col:
            
            yield item
            
def append_all(q, iterable):
    """ Append all items in the iterable to the queue. """
    
    for item in iterable: q.append(item)
        
def product_limiter():
    """
    This helps trim the library a bit by excluding rates with
    products with more protons than Ni, heavier than Ni56,
    or with relatively high / low neutron percentages. 
    """
    
    # Proton number bounds
    Zlo, Zhi = 6, endpoint.Z
    # Nucleon number bounds
    Alo, Ahi = 12, endpoint.A
    # Bounds on A / Z ratio to drop peripheral nuclei
    Rlo, Rhi = 1.69, 2.2
    
    def limit_products(r):
        
        meet_conds = \
        (
            (Zlo <= p.Z <= Zhi and
            Alo <= p.A <= Ahi and
            Rlo <= p.A / p.Z <= Rhi and
            (p.N, p.Z) in bintable.energies)
            or (p.Z, p.A) == (1, 1)
            or (p.Z, p.A) == (2, 4)
            for p in r.products
        )
        return all(meet_conds)
    
    return limit_products

In [10]:
limiter = product_limiter()
final_lib = Library(rates=core_lib.get_rates())

seeds = [nuc for nuc in core_nuclei if nuc.A >= 12]
encountered = set(seeds) | {Nucleus("p"), Nucleus("he4")}
seeds = deque(seeds)

while seeds:
    
    # Get the new rates with seed as a reactant
    seed = seeds.popleft()
    filt = RateFilter(reactants=seed, filter_function=limiter, exact=False)
    new_lib = red_lib.filter(filt)
    if new_lib is None: continue
    final_lib += new_lib
    
    # Append all unseen nuclei to the queue
    prod = (r.products for r in new_lib.get_rates())
    prod = flatten(prod)
    prod = filter(lambda p: p not in encountered, prod)
    prod = sorted(set(prod))
    append_all(seeds, prod)
    encountered.update(prod)
    
encountered = sorted(encountered)

In [11]:
final_lib

p + p + e⁻ ⟶ H2 + 𝜈            [Q =   1.44 MeV] (p + p --> d <ec_reaclib_weak_>)
p + p ⟶ H2 + e⁺ + 𝜈            [Q =   1.44 MeV] (p + p --> d <bet+_reaclib_weak_>)
H2 + p ⟶ He3 + 𝛾               [Q =   5.49 MeV] (d + p --> he3 <de04_reaclib__>)
H2 + H2 ⟶ He4 + 𝛾              [Q =  23.85 MeV] (d + d --> he4 <nacr_reaclib__>)
He3 + p ⟶ He4 + e⁺ + 𝜈         [Q =  19.80 MeV] (he3 + p --> he4 <bet+_reaclib_weak_>)
He3 + H2 ⟶ p + He4             [Q =  18.35 MeV] (he3 + d --> p + he4 <de04_reaclib__>)
He3 + He3 ⟶ p + p + He4        [Q =  12.86 MeV] (he3 + he3 --> p + p + he4 <nacr_reaclib__>)
He3 + He4 ⟶ Be7 + 𝛾            [Q =   1.59 MeV] (he3 + he4 --> be7 <cd08_reaclib__>)
He4 + He4 + He4 ⟶ C12 + 𝛾      [Q =   7.28 MeV] (he4 + he4 + he4 --> c12 <fy05_reaclib__>)
Li7 + p ⟶ He4 + He4            [Q =  17.35 MeV] (li7 + p --> he4 + he4 <de04_reaclib__>)
Be7 + p ⟶ B8 + 𝛾               [Q =   0.14 MeV] (be7 + p --> b8 <nacr_reaclib__>)
Be7 + H2 ⟶ p + He4 + He4       [Q =  16.77 MeV] (be7 + d -->

In [12]:
encountered

[p,
 he4,
 c12,
 c13,
 n12,
 n13,
 n14,
 n15,
 o14,
 o15,
 o16,
 o17,
 o18,
 f16,
 f17,
 f18,
 f19,
 f20,
 ne17,
 ne18,
 ne19,
 ne20,
 ne21,
 ne22,
 na19,
 na20,
 na21,
 na22,
 na23,
 na24,
 mg21,
 mg22,
 mg23,
 mg24,
 mg25,
 mg26,
 al22,
 al23,
 al24,
 al25,
 al26,
 al27,
 al28,
 si24,
 si25,
 si26,
 si27,
 si28,
 si29,
 si30,
 p26,
 p27,
 p28,
 p29,
 p30,
 p31,
 p32,
 p33,
 s28,
 s29,
 s30,
 s31,
 s32,
 s33,
 s34,
 s35,
 cl29,
 cl30,
 cl31,
 cl32,
 cl33,
 cl34,
 cl35,
 cl36,
 cl37,
 ar31,
 ar32,
 ar33,
 ar34,
 ar35,
 ar36,
 ar37,
 ar38,
 ar39,
 k33,
 k34,
 k35,
 k36,
 k37,
 k38,
 k39,
 k40,
 k41,
 ca34,
 ca35,
 ca36,
 ca37,
 ca38,
 ca39,
 ca40,
 ca41,
 ca42,
 ca43,
 ca44,
 sc36,
 sc37,
 sc38,
 sc39,
 sc40,
 sc41,
 sc42,
 sc43,
 sc44,
 sc45,
 sc46,
 ti38,
 ti39,
 ti40,
 ti41,
 ti42,
 ti43,
 ti44,
 ti45,
 ti46,
 ti47,
 ti48,
 v40,
 v41,
 v42,
 v43,
 v44,
 v45,
 v46,
 v47,
 v48,
 v49,
 v50,
 cr42,
 cr43,
 cr44,
 cr45,
 cr46,
 cr47,
 cr48,
 cr49,
 cr50,
 cr51,
 cr52,
 mn44,
 mn45,
 mn46,

In [13]:
rp_net = PythonNetwork(libraries=[final_lib])

In [14]:
# rp_net.write_network()